In [2]:
import os
import pandas as pd

In [3]:
mcv_dir = '/mnt/data/mozilla_common_voice/en_1488h_2019-12-10'
train = os.path.join(mcv_dir, 'train.tsv')
dev = os.path.join(mcv_dir, 'dev.tsv')
test = os.path.join(mcv_dir, 'test.tsv')

In [3]:
train_df = pd.read_csv(train, sep='\t')
train_df['partition'] = 'train'

dev_df = pd.read_csv(dev, sep='\t')
dev_df['partition'] = 'dev'

test_df = pd.read_csv(test, sep='\t')
test_df['partition'] = 'test'

all_df = pd.concat([train_df, dev_df, test_df])

In [4]:
# load cached audio_dur
train_df = pd.read_csv(os.path.join(mcv_dir, 'train_audio.csv'))
train_df['partition'] = 'train'
dev_df = pd.read_csv(os.path.join(mcv_dir, 'dev_audio.csv'))
dev_df['partition'] = 'dev'
test_df = pd.read_csv(os.path.join(mcv_dir, 'test_audio.csv'))
test_df['partition'] = 'test'
all_df = pd.concat([train_df, dev_df, test_df])

/home/s1462938/.conda/envs/mcv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Average utterances per speaker

In [110]:
all_utts = len(train_df)
speakers = len(train_df['client_id'].unique())
accent_utts = len(train_df[~train_df['accent'].isnull()])
accent_speakers = len(train_df[~train_df['accent'].isnull()]['client_id'].unique())

print(f"""Total train utts: {all_utts}
Utts with accent labels: {accent_utts}

Unique speakers: {speakers}
Speakers with accent labels: {accent_speakers}""")

Total train utts: 232975
Utts with accent labels: 135391

Unique speakers: 10013
Speakers with accent labels: 3220


In [31]:
all_df.groupby('partition').agg(
    {'client_id': lambda x:len(x) / len(x.unique())}
)

client_id
partition           
dev         3.551566
test        1.639675
train      23.267253

In [32]:
dev_df['client_id'].value_counts().value_counts().sort_index(ascending=False)

5     871
4    2146
3     460
2     316
1     580
Name: client_id, dtype: int64

In [33]:
test_df['client_id'].value_counts().value_counts().sort_index(ascending=False)

3    1546
2    2967
1    4959
Name: client_id, dtype: int64

Many speakers in train set with lots of utterances.

In [34]:
(train_df['client_id'].value_counts() > 100).value_counts()

False    9654
True      359
Name: client_id, dtype: int64

In [35]:
train_df['client_id'].value_counts().value_counts().sort_index(ascending=False).head()

6441    1
4234    1
2772    1
2693    1
2681    1
Name: client_id, dtype: int64

In [36]:
train_df['client_id'].value_counts().head()

ab72e9ab22713aec03a3189202a0713e56016ea07569b3041fcd65d77c9eb3f8c692f7ccaec75c16bbc62476d528f434d829193994cf07fcde9e3b1e4a2fa93a    6441
7963691c43c8cc498c58f117527522bf772c76c38530570bc55ef04834f67fb7a9227bd0fa1f13e64e8de1cde6594f3501e172ab86559697c08726cac26f4c6f    4234
7eff9a54bdb0619deffda7609d5b8565278e3328de99e63908be2a37c1ad16516240b9c92bc6b68d8130b20dc556f57005dc053f4874a49589f5971a31b97e98    2772
e6dbbe39377aa0fe2851852b2c695c9c31094c80c9ccd244bac8dfdc8a523b2a114362d4c717bd9d628499fe1d3c11971221be5ec4dadce15c9ff8c5254ae368    2693
29b8505586cd43382cd695da6b943f401104be710a5b60e814ac5fe7e06b39459cf8fe1701ca83f8154b3ccd749df7c2aef33ff23950bb1a135b1e1c393dbcf6    2681
Name: client_id, dtype: int64

In [5]:
small_train = train_df.groupby('client_id').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)

In [6]:
small_train[['accent', 'partition']].pivot_table(
    index='accent', columns='partition',
    aggfunc=len, fill_value=0, margins=True
).sort_values('All', ascending=False)

partition       train    All
accent                      
All             76429  76429
us              39701  39701
england         11672  11672
indian           8641   8641
canada           5598   5598
australia        4697   4697
african          1192   1192
ireland          1084   1084
scotland          987    987
newzealand        969    969
philippines       691    691
singapore         392    392
malaysia          332    332
wales             191    191
other             169    169
hongkong           73     73
bermuda            38     38
southatlandtic      2      2

In [7]:
small_utts = len(small_train)
small_speakers = len(small_train['client_id'].unique())
small_accent_utts = len(small_train[~small_train['accent'].isnull()])
small_accent_speakers = len(small_train[~small_train['accent'].isnull()]['client_id'].unique())

print(f"""Total train utts: {small_utts}
Utts with accent labels: {small_accent_utts}

Unique speakers: {small_speakers}
Speakers with accent labels: {small_accent_speakers}""")

Total train utts: 152892
Utts with accent labels: 76429

Unique speakers: 10013
Speakers with accent labels: 3220


In [40]:
small_train.groupby('accent').agg(
    {'client_id': lambda x:len(x) / len(x.unique())}
)

client_id
accent                   
african         24.326531
australia       29.727848
bermuda          7.600000
canada          26.784689
england         23.437751
hongkong         8.111111
indian          22.328165
ireland         27.100000
malaysia        14.434783
newzealand      27.685714
other            7.041667
philippines     27.640000
scotland        24.073171
singapore       26.133333
southatlandtic   2.000000
us              23.561424
wales           11.937500

### Accent utterance counts per partition

In [136]:
len(train_df[~train_df['accent'].isnull()]) / len(train_df)

0.5811396072539972

In [111]:
all_df[['accent', 'partition']].pivot_table(
    index='accent', columns='partition',
    aggfunc=len, fill_value=0, margins=True,
).sort_values('All', ascending=False)

partition        dev  test   train     All
accent                                    
All             2100  1398  135391  138889
us              1048   669   72392   74109
england          304   153   19204   19661
australia         85    28   12208   12321
indian           324   314   11395   12033
canada           114    63    9150    9327
scotland           7    14    3686    3707
newzealand        31    13    1949    1993
ireland           33    18    1866    1917
african           40    26    1326    1392
philippines       18    15     834     867
singapore         16     6     576     598
malaysia           8    11     332     351
other             20    40     169     229
wales             31     0     191     222
hongkong          17    13      73     103
bermuda            4    12      38      54
southatlandtic     0     3       2       5

### Accent and gender counts per partition

In [42]:
train_df[['accent', 'gender']].pivot_table(
    index='accent', columns='gender',
    aggfunc=len, fill_value=0, margins=True
).sort_values('All', ascending=False)

gender          female   male  other     All
accent                                      
All              29444  97992   6340  133776
us               16593  53118   1238   70949
england           3346  14978    811   19135
australia          942   6953   4269   12164
indian            1609   9776      0   11385
canada            2588   6545      5    9138
scotland          2950    736      0    3686
newzealand         261   1658      0    1919
ireland            562   1294     10    1866
african            238   1088      0    1326
philippines        113    721      0     834
singapore           34    542      0     576
malaysia            62    270      0     332
wales              111     73      7     191
other                6    156      0     162
hongkong            28     45      0      73
bermuda              1     37      0      38
southatlandtic       0      2      0       2

In [43]:
dev_df[['accent', 'gender']].pivot_table(
    index='accent', columns='gender',
    aggfunc=len, fill_value=0, margins=True
).sort_values('All', ascending=False)

gender       female  male  other   All
accent                                
All             414  1648      7  2069
us              246   783      4  1033
indian            9   311      0   320
england          43   254      3   300
canada           36    74      0   110
australia        21    64      0    85
african           5    35      0    40
newzealand        8    23      0    31
wales            14    17      0    31
ireland           4    25      0    29
other             4    16      0    20
philippines       8    10      0    18
hongkong          8     9      0    17
singapore         0    16      0    16
malaysia          4     4      0     8
scotland          0     7      0     7
bermuda           4     0      0     4

In [44]:
test_df[['accent', 'gender']].pivot_table(
    index='accent', columns='gender',
    aggfunc=len, fill_value=0, margins=True
).sort_values('All', ascending=False)

gender          female  male  other   All
accent                                   
All                227  1141     18  1386
us                 150   509      5   664
indian              18   294      0   312
england             18   130      4   152
canada               7    49      6    62
other                2    37      1    40
african              5    21      0    26
australia            8    17      0    25
ireland              3    13      2    18
philippines          3    12      0    15
scotland             2    12      0    14
newzealand           1    12      0    13
hongkong             4     9      0    13
bermuda              3     9      0    12
malaysia             3     8      0    11
singapore            0     6      0     6
southatlandtic       0     3      0     3

### Transcriptions

In [8]:
import string

train_split_text = train_df['sentence'].str.lower().apply(
    lambda x: x.translate(str.maketrans('', '', string.punctuation))
).str.split(' ')

In [9]:
tokens = train_split_text.agg(len).sum()
types = set()
train_split_text.apply(lambda x: types.update(x))
print(f"""Total tokens: {tokens}
Total types: {len(types)}""")

Total tokens: 2379928
Total types: 124677


In [47]:
# overlap with dev/test

### Audio metrics

In [10]:

audio_dir = os.path.join(mcv_dir, 'clips')

# avg utterance length
# summed per speaker
# per accent
# per partition

In [11]:
import glob

audio_files = glob.glob(f"{audio_dir}/*")

In [78]:
all_df['path'].apply(lambda x: os.path.exists(os.path.join(audio_dir, x))).sum() == len(all_df)

True

In [12]:
import sys
from mutagen.mp3 import MP3

def get_mp3_dur(f, audio_dir=None):
    """Return duration in seconds of mp3 file"""
    if audio_dir is not None:
        f = os.path.join(audio_dir, f)
    try:
        a = MP3(f)
        return a.info.length
    except Exception as e:
        return repr(e)

In [65]:
train_df['path'].head().apply(lambda x: get_mp3_dur(x, audio_dir)).sum()

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 17.2 µs


27.456

In [99]:
%%time
x = test_df['path'].head()
x[0] = 'blah.mp3'
y = x.apply(lambda x: get_mp3_dur(x, audio_dir))

CPU times: user 2.33 ms, sys: 275 µs, total: 2.6 ms
Wall time: 3.87 ms


In [13]:
%%time
all_df['audio_dur'] = all_df['path'].apply(lambda x: get_mp3_dur(x, audio_dir))

CPU times: user 3min 15s, sys: 32.2 s, total: 3min 48s
Wall time: 18min 20s


In [152]:
# seem like empty files
# these are all recordings from a single client_id, consecutive filenames => single session?
# that person has 650 other recordings so no worries just to drop these
all_df[all_df['audio_dur'].map(type) != float]

client_id  \
185141  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185142  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185143  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185144  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185145  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185146  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185147  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185148  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185149  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185150  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185151  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185152  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185153  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185154  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185155  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185156  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   
185158  dc3db1a53620d85b9664bf89a66c12905ad39c9c4cd2bf...   

                                path  \
185141  common_voice_en_19411965.mp3   
185142  common_voice_en_19411966.mp3   
185143  common_voice_en_19411967.mp3   
185144  common_voice_en_19411968.mp3   
185145  common_voice_en_19411974.mp3   
185146  common_voice_en_19411975.mp3   
185147  common_voice_en_19411976.mp3   
185148  common_voice_en_19411977.mp3   
185149  common_voice_en_19411978.mp3   
185150  common_voice_en_19411984.mp3   
185151  common_voice_en_19411985.mp3   
185152  common_voice_en_19411986.mp3   
185153  common_voice_en_19411987.mp3   
185154  common_voice_en_19411988.mp3   
185155  common_voice_en_19411989.mp3   
185156  common_voice_en_19411990.mp3   
185158  common_voice_en_19411992.mp3   

                                                 sentence  up_votes  \
185141  They possessed middle and rear back vertebrae ...         2   
185142      Chairman of the Gateway Group till his death.         2   
185143     The team played its home games at Fenway Park.         2   
185144  This team consists of Fireball, Kuei, Phantasm...         2   
185145  Murray tried to restrain the MacDonalds, but t...         2   
185146  It may be the remnant of the core of a larger,...         2   
185147  This enzyme is extremely important in the proc...         2   
185148      Josceline's son was Reginald, bishop of Bath.         2   
185149  Many of the students are either international ...         2   
185150   She told them that there is no reality in signs.         2   
185151  Linacre's only medical works were his translat...         2   
185152  During the conversation another piece of horse...         2   
185153            The Primary school is on Headland Road.         2   
185154  The miners' demands for economic and political...         2   
185155  Shibajuku Station has a single side platform s...         2   
185156  If one break a man's bone, they shall break hi...         2   
185158  A fitness contest, the Kike Elomaa Fitness Cha...         2   

        down_votes       age gender   accent partition  \
185141           0  thirties  other  england     train   
185142           0  thirties  other  england     train   
185143           0  thirties  other  england     train   
185144           0  thirties  other  england     train   
185145           0  thirties  other  england     train   
185146           0  thirties  other  england     train   
185147           0  thirties  other  england     train   
185148           0  thirties  other  england     train   
185149           0  thirties  other  england     train   
185150           0  thirties  other  england     train   
185151           0  thirties  other  england     train   
185152           0  thirties  other  england     train   
185153           0  thirties  other  england     train   
185154           0  thirties  other  england     train   
185155           0  thirties  other  england     train   
185156           0  t

In [ ]:
train_df['audio_dur'] = train_df['path'].apply(lambda x: get_mp3_dur(x, audio_dir))
dev_df['audio_dur'] = dev_df['path'].apply(lambda x: get_mp3_dur(x, audio_dir))
test_df['audio_dur'] = test_df['path'].apply(lambda x: get_mp3_dur(x, audio_dir))

In [22]:
# cache these expensive things...
train_df.to_csv(os.path.join(mcv_dir, 'train_audio.csv'), index=False)
dev_df.to_csv(os.path.join(mcv_dir, 'dev_audio.csv'), index=False)
test_df.to_csv(os.path.join(mcv_dir, 'test_audio.csv'), index=False)

In [5]:
print(f"Train data: {train_df[train_df['audio_dur'].map(type) == float]['audio_dur'].sum() / 3600.:.1f}h")
print(f"Dev data: {dev_df[dev_df['audio_dur'].map(type) == float]['audio_dur'].sum() / 3600.:.1f}h")
print(f"Test data: {test_df[test_df['audio_dur'].map(type) == float]['audio_dur'].sum() / 3600.:.1f}h")

Train data: 263.6h
Dev data: 26.1h
Test data: 24.7h


In [142]:
# hours of audio per accent per partition

all_df[all_df['audio_dur'].map(type) == float].pivot_table(
    index='accent', columns='partition', values='audio_dur',
    aggfunc=sum, fill_value=0, margins=True,
) / 3600

partition            dev      test       train         All
accent                                                    
african         0.062347  0.044400    2.126400    2.233147
australia       0.141840  0.040553   18.603627   18.786020
bermuda         0.007393  0.012740    0.065320    0.085453
canada          0.184100  0.130180   14.184707   14.498987
england         0.504720  0.239787   28.513049   29.257556
hongkong        0.029373  0.017587    0.118540    0.165500
indian          0.535280  0.499613   18.324633   19.359527
ireland         0.048613  0.026333    2.924753    2.999700
malaysia        0.013693  0.016700    0.529267    0.559660
newzealand      0.054793  0.022213    3.203592    3.280598
other           0.027980  0.048820    0.220980    0.297780
philippines     0.028573  0.031593    1.220553    1.280720
scotland        0.011160  0.022360    6.613515    6.647035
singapore       0.026593  0.010340    0.900227    0.937160
southatlandtic  0.000000  0.005200    0.002273    0.007473
us              1.705993  1.033733  109.553956  112.293683
wales           0.055053  0.000000    0.298413    0.353467
All             3.437507  2.202153  207.403807  213.043467

In [12]:
import datetime

def pretty_time_agg(series):
    s = int(series.sum())
    m = s // 60
    h = m // 60
    #d = h // 24
    #h = h % 24
    m = m % 60
    s = s % 60
    #if d > 0:
    #    return "{0}d {1}".format(d, datetime.time(h, m, s).strftime("%Hh %Mm %Ss"))
    #else:
    #    return datetime.time(h, m, s).strftime("%Hh %Mm %Ss")
    return "{}h {}m {}s".format(h, m, s)

all_df[all_df['audio_dur'].map(type) == float].pivot_table(
    index='accent', columns='partition', values='audio_dur',
    aggfunc=pretty_time_agg, fill_value=0, margins=True,
)

partition              dev        test         train           All
accent                                                            
african          0h 3m 44s   0h 2m 39s    1h 45m 28s    1h 51m 53s
australia        0h 8m 30s   0h 2m 25s    12h 30m 5s    12h 41m 1s
bermuda          0h 0m 26s   0h 0m 45s     0h 3m 55s      0h 5m 7s
canada           0h 11m 2s   0h 7m 48s     8h 8m 56s    8h 27m 47s
england         0h 30m 16s  0h 14m 23s   19h 24m 43s    20h 9m 24s
hongkong         0h 1m 45s    0h 1m 3s      0h 7m 6s     0h 9m 55s
indian           0h 32m 7s  0h 29m 58s   12h 21m 45s   13h 23m 51s
ireland          0h 2m 55s   0h 1m 34s    0h 58m 49s     1h 3m 19s
malaysia         0h 0m 49s    0h 1m 0s    0h 31m 45s    0h 33m 34s
newzealand       0h 3m 17s   0h 1m 19s     0h 59m 7s     1h 3m 44s
other            0h 1m 40s   0h 2m 55s      0h 7m 4s    0h 11m 41s
philippines      0h 1m 42s   0h 1m 53s    0h 58m 10s     1h 1m 47s
scotland         0h 0m 40s   0h 1m 20s    6h 21m 39s    6h 23m 40s
singapore        0h 1m 35s   0h 0m 37s    0h 26m 38s    0h 28m 51s
southatlandtic           0   0h 0m 18s      0h 0m 8s     0h 0m 26s
us              1h 42m 21s    1h 2m 1s   73h 15m 39s     76h 0m 2s
wales            0h 3m 18s           0    0h 17m 49s     0h 21m 7s
All             3h 26m 15s   2h 12m 7s  138h 18m 55s  143h 57m 18s

### Validated data
Around 30% of validated data has been included in official partitions, but all of this has been through the same level of review.

In [112]:
valid = os.path.join(mcv_dir, 'validated.tsv')
valid_df = pd.read_csv(valid, sep='\t')
full_valid_len = len(valid_df)

/home/s1462938/.conda/envs/torch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [113]:
%%time
valid_df['partition'] = 'valid'
valid_df['audio_dur'] = valid_df['path'].apply(lambda x: get_mp3_dur(x, audio_dir))
valid_df.to_csv(os.path.join(mcv_dir, 'validated_audio.csv'), index=False)

KeyboardInterrupt: 

In [114]:
# filter by client_id so no speaker overlap between splits
valid_df = valid_df[~valid_df['client_id'].isin(all_df['client_id'])]

# filtering by filename actually gets the expected length
# => not all validated utterances per speaker selected for partition data
#valid_df = valid_df[~valid_df['path'].isin(all_df['path'])]
#len(valid_df) == full_valid_len - len(train_df) - len(dev_df) - len(test_df)

In [115]:
len(valid_df[~valid_df['accent'].isnull()]) / len(valid_df)

0.4257476432681499

In [116]:
all_valid_df = pd.concat([all_df, valid_df])
all_valid_df[all_valid_df['audio_dur'].map(type) == float].pivot_table(
    index='accent', columns='partition', values='audio_dur',
    aggfunc=pretty_time_agg, fill_value=0, margins=True,
)

partition               dev         test           train        valid  \
accent                                                                  
african         00h 03m 44s  00h 02m 39s     01h 45m 28s  00h 00m 00s   
australia       00h 08m 30s  00h 02m 25s     12h 30m 05s  00h 00m 00s   
bermuda         00h 00m 26s  00h 00m 45s     00h 03m 55s  00h 00m 00s   
canada          00h 11m 02s  00h 07m 48s     08h 08m 56s  00h 00m 00s   
england         00h 30m 16s  00h 14m 23s     19h 24m 43s  00h 00m 00s   
hongkong        00h 01m 45s  00h 01m 03s     00h 07m 06s  00h 00m 00s   
indian          00h 32m 07s  00h 29m 58s     12h 21m 45s  00h 00m 00s   
ireland         00h 02m 55s  00h 01m 34s     00h 58m 49s  00h 00m 00s   
malaysia        00h 00m 49s  00h 01m 00s     00h 31m 45s  00h 00m 00s   
newzealand      00h 03m 17s  00h 01m 19s     00h 59m 07s  00h 00m 00s   
other           00h 01m 40s  00h 02m 55s     00h 07m 04s  00h 00m 00s   
philippines     00h 01m 42s  00h 01m 53s     00h 58m 10s  00h 00m 00s   
scotland        00h 00m 40s  00h 01m 20s     06h 21m 39s  00h 00m 00s   
singapore       00h 01m 35s  00h 00m 37s     00h 26m 38s  00h 00m 00s   
southatlandtic            0  00h 00m 18s     00h 00m 08s  00h 00m 00s   
us              01h 42m 21s  01h 02m 01s  3d 01h 15m 39s  00h 00m 00s   
wales           00h 03m 18s            0     00h 17m 49s  00h 00m 00s   
All             03h 26m 15s  02h 12m 07s  5d 18h 18m 55s            0   

partition                  All  
accent                          
african            01h 51m 53s  
australia          12h 41m 01s  
bermuda            00h 05m 07s  
canada             08h 27m 47s  
england            20h 09m 24s  
hongkong           00h 09m 55s  
indian             13h 23m 51s  
ireland            01h 03m 19s  
malaysia           00h 33m 34s  
newzealand         01h 03m 44s  
other              00h 11m 41s  
philippines        01h 01m 47s  
scotland           06h 23m 40s  
singapore          00h 28m 51s  
southatlandtic     00h 00m 26s  
us              3d 04h 00m 02s  
wales              00h 21m 07s  
All             5d 23h 57m 18s

In [202]:
all_valid_df[all_valid_df['audio_dur'].map(type) == float][['accent', 'partition']].pivot_table(
    index='accent', columns='partition',
    aggfunc=len, fill_value=0, margins=True,
)

partition        dev  test   train   valid     All
accent                                            
african           40    26    1326    2975    4367
australia         85    28   12208   11574   23895
bermuda            4    12      38     415     469
canada           114    63    9150    9285   18612
england          304   153   19187   34727   54371
hongkong          17    13      73    1029    1132
indian           324   314   11395   14913   26946
ireland           33    18    1866    2979    4896
malaysia           8    11     332     580     931
newzealand        31    13    1949    6696    8689
other             20    40     169    7679    7908
philippines       18    15     834     808    1675
scotland           7    14    3686    3003    6710
singapore         16     6     576     251     849
southatlandtic     0     3       2     200     205
us              1048   669   72392   93262  167371
wales             31     0     191     889    1111
All             2100  1398  135374  191265  330137

In [30]:
print(f"Validation utts from new speakers: {len(valid_df)}/{full_valid_len}")
print(f"New speakers: {len(valid_df['client_id'].unique())}")

Validation utts from new speakers: 449245/854444
New speakers: 19768


In [49]:
seen_transcripts = pd.concat([train_df, dev_df, test_df])['sentence']
valid_df_uniq_text = valid_df[~valid_df['sentence'].isin(seen_transcripts)]
# complete overlap of validated utt transcripts with those seen in other partitions
print(f"Validated utts with new transcripts: {len(valid_df_uniq_text)}")
print(f"Train transcripts seen in validated data: {sum(train_df['sentence'].isin(valid_df['sentence']))}/{len(train_df)}")
print(f"Dev transcripts seen in validated data: {sum(dev_df['sentence'].isin(valid_df['sentence']))}/{len(dev_df)}")
print(f"Test transcripts seen in validated data: {sum(test_df['sentence'].isin(valid_df['sentence']))}/{len(test_df)}")
# these add up the same
print(f"Unique transcripts in validated data: {len(valid_df['sentence'].unique())}")

Validated utts with new transcripts: 0
Train transcripts seen in validated data: 2643/232975
Dev transcripts seen in validated data: 1372/15531
Test transcripts seen in validated data: 8009/15531
Unique transcripts in validated data: 12024


In [92]:
valid_df = pd.read_csv(os.path.join(mcv_dir, 'validated_audio.csv'))

In [93]:
# okay so limited to same transcripts as test set, but that's fine we just throw it away
# and maybe can maximise the amount of accent information in new test set
valid_test = valid_df[valid_df['sentence'].isin(test_df['sentence'])]
valid_test = pd.concat([valid_test, test_df])
print(f"Validated utts with test transcripts and accent labels: {len(valid_test[~valid_test['accent'].isnull()]['sentence'].unique())}/{len(valid_test['sentence'].unique())}")
print(f"vs. Test utts with accent labels: {len(test_df[~test_df['accent'].isnull()]['sentence'].unique())}/{len(test_df)}")

Validated utts with test transcripts and accent labels: 8617/15531
vs. Test utts with accent labels: 1398/15531


In [94]:
valid_test_accent = pd.concat([valid_test[~(valid_test['accent'].isnull()) & (valid_test['accent']!='other')].groupby('sentence').apply(lambda x:x.sample(1))])
valid_test_no_accent = pd.concat([valid_test[~valid_test['sentence'].isin(valid_test_accent['sentence'])].groupby('sentence').apply(lambda x:x.sample(1))])
valid_test = pd.concat([valid_test_accent, valid_test_no_accent])

In [77]:
valid_test['partition'] = 'valid'
valid_test['audio_dur'] = valid_test['path'].apply(lambda x: get_mp3_dur(x, audio_dir))

In [95]:
valid_test.to_csv(os.path.join(mcv_dir, 'validated_test_audio.csv'), index=False)

In [96]:
valid_vs_test = pd.concat([valid_test, test_df])
valid_vs_test[valid_vs_test['audio_dur'].map(type) == float].pivot_table(
    index='accent', columns='partition', values='audio_dur',
    aggfunc=pretty_time_agg, fill_value=0, margins=True,
)

partition              test        valid          All
accent                                               
african         00h 04m 39s  00h 08m 23s  00h 13m 02s
australia       00h 04m 05s  00h 31m 29s  00h 35m 34s
bermuda         00h 00m 52s  00h 01m 17s  00h 02m 09s
canada          00h 10m 53s  00h 22m 53s  00h 33m 46s
england         00h 23m 39s  01h 34m 47s  01h 58m 27s
hongkong        00h 01m 33s  00h 02m 50s  00h 04m 24s
indian          00h 48m 57s  00h 42m 22s  01h 31m 19s
ireland         00h 02m 38s  00h 07m 46s  00h 10m 24s
malaysia        00h 01m 36s  00h 01m 32s  00h 03m 08s
newzealand      00h 02m 17s  00h 21m 17s  00h 23m 34s
other           00h 02m 55s  00h 00m 04s  00h 03m 00s
philippines     00h 03m 47s  00h 02m 34s  00h 06m 22s
scotland        00h 01m 45s  00h 07m 02s  00h 08m 47s
singapore       00h 01m 08s  00h 00m 42s  00h 01m 50s
southatlandtic  00h 00m 37s  00h 00m 23s  00h 01m 00s
us              01h 41m 07s  04h 16m 47s  05h 57m 55s
wales                     0  00h 01m 38s  00h 01m 38s
All             03h 32m 34s  08h 23m 53s  11h 56m 28s

In [98]:
valid_vs_test = pd.concat([valid_test, test_df])
valid_vs_test[valid_vs_test['audio_dur'].map(type) == float].pivot_table(
    index='accent', columns='partition', values='audio_dur',
    aggfunc=len, fill_value=0, margins=True,
)

partition       test  valid    All
accent                            
african           44    122    166
australia         44    487    531
bermuda           13     17     30
canada            95    349    444
england          244   1457   1701
hongkong          19     46     65
indian           494    644   1138
ireland           30    126    156
malaysia          17     22     39
newzealand        22    306    328
other             40      1     41
philippines       30     35     65
scotland          17    122    139
singapore         11      9     20
southatlandtic     6      7     13
us              1056   4054   5110
wales              0     29     29
All             2182   7833  10015

### Resampled using [CorporaCreator](https://github.com/mozilla/CorporaCreator)

In [6]:
all_audio_durs = pd.concat([pd.read_csv(os.path.join(mcv_dir, i)) for i in ['train_audio.csv', 'dev_audio.csv', 'test_audio.csv', 'validated_audio.csv']])
all_audio_durs = all_audio_durs.set_index('path')

train_df_new = pd.read_csv(os.path.join(mcv_dir, 'accent_meta', 'en', 'train.tsv'), sep='\t')
train_df_new['partition'] = 'train'
train_df_new = train_df_new.set_index('path')
train_df_new['audio_dur'] = all_audio_durs['audio_dur']
train_df_new = train_df_new.reset_index()

dev_df_new = pd.read_csv(os.path.join(mcv_dir, 'accent_meta', 'en', 'dev.tsv'), sep='\t')
dev_df_new['partition'] = 'dev'
dev_df_new = dev_df_new.set_index('path')
dev_df_new['audio_dur'] = all_audio_durs['audio_dur']
dev_df_new = dev_df_new.reset_index()

test_df_new = pd.read_csv(os.path.join(mcv_dir, 'accent_meta', 'en', 'test.tsv'), sep='\t')
test_df_new['partition'] = 'test'
test_df_new = test_df_new.set_index('path')
test_df_new['audio_dur'] = all_audio_durs['audio_dur']
test_df_new = test_df_new.reset_index()

all_new_parts = pd.concat([train_df_new, dev_df_new, test_df_new])

/home/s1462938/.conda/envs/mcv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [119]:
all_new_parts['audio_dur'] = all_new_parts['path'].apply(lambda x: get_mp3_dur(x, audio_dir))

In [17]:
all_new_parts.pivot_table(
    index='accent', columns='partition', values='path',
    aggfunc=len, fill_value=0, margins=True,
)

partition         dev   test   train     All
accent                                      
african           254    273    1327    1854
australia         825    529   12101   13455
bermuda            22     57       2      81
canada            812    820    8885   10517
england          2510   2120   18425   23055
hongkong           57     80      19     156
indian           1903   2283    9932   14118
ireland           222    201    1641    2064
malaysia          113    115     252     480
newzealand         87    155    1923    2165
philippines       151    146     721    1018
scotland          205    151    3734    4090
singapore          81     54     516     651
southatlandtic      0     13       7      20
us               7476   7631   69163   84270
wales              18    108     155     281
All             14736  14736  128803  158275

In [14]:
all_new_parts[all_new_parts['audio_dur'].map(type) == float].pivot_table(
    index='accent', columns='partition', values='audio_dur',
    aggfunc=pretty_time_agg, fill_value=0, margins=True,
)

partition              dev        test         train          All
accent                                                           
african          0h 22m 5s  0h 24m 46s     1h 19m 0s    2h 5m 51s
australia        1h 9m 36s  0h 44m 17s   11h 16m 56s  13h 10m 50s
bermuda           0h 2m 8s   0h 4m 15s      0h 0m 7s    0h 6m 31s
canada           1h 3m 45s   1h 6m 45s     6h 45m 7s   8h 55m 38s
england          3h 30m 4s  2h 54m 13s    15h 19m 1s  21h 43m 20s
hongkong          0h 5m 3s   0h 6m 35s      0h 1m 9s   0h 12m 48s
indian          2h 41m 30s   3h 5m 41s    8h 54m 25s  14h 41m 37s
ireland         0h 18m 30s  0h 22m 15s     0h 36m 7s   1h 16m 53s
malaysia        0h 10m 23s   0h 9m 29s     0h 18m 9s    0h 38m 1s
newzealand       0h 5m 44s  0h 11m 29s    0h 51m 51s     1h 9m 5s
philippines     0h 13m 14s  0h 12m 21s     0h 44m 3s    1h 9m 39s
scotland        0h 18m 24s  0h 12m 21s    5h 58m 54s   6h 29m 39s
singapore        0h 7m 35s   0h 4m 59s    0h 17m 45s   0h 30m 20s
southatlandtic           0    0h 1m 3s     0h 0m 25s    0h 1m 28s
us              9h 58m 39s  10h 12m 5s   60h 17m 30s  80h 28m 15s
wales            0h 1m 49s   0h 8m 58s    0h 12m 20s    0h 23m 9s
All             20h 8m 39s  20h 1m 38s  112h 52m 56s  153h 3m 14s

In [122]:
train_split_text_new = train_df_new['sentence'].str.lower().apply(
    lambda x: x.translate(str.maketrans('', '', string.punctuation))
).str.split(' ')

In [124]:
tokens = train_split_text_new.agg(len).sum()
types = set()
train_split_text_new.apply(lambda x: types.update(x))
print(f"""Total tokens: {tokens}
Total types: {len(types)}""")

Total tokens: 1304623
Total types: 85805
